In [5]:
!mkdir data/ 
!wget https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar -O data/ILSVRC2012_img_val.tar
!tar -xf data/ILSVRC2012_img_val.tar -C data/
!wget https://raw.githubusercontent.com/Evolving-AI-Lab/ppgn/master/misc/map_clsloc.txt

mkdir: cannot create directory ‘data/’: File exists
--2022-04-30 02:33:46--  https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar
Resolving image-net.org (image-net.org)... 171.64.68.16
Connecting to image-net.org (image-net.org)|171.64.68.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6744924160 (6.3G) [application/x-tar]
Saving to: ‘data/ILSVRC2012_img_val.tar’

data/ILSVRC2012_img 100%[===================>]   6.28G  19.5MB/s    in 7m 47s  

2022-04-30 02:41:34 (13.8 MB/s) - ‘data/ILSVRC2012_img_val.tar’ saved [6744924160/6744924160]

--2022-04-30 02:42:26--  https://raw.githubusercontent.com/Evolving-AI-Lab/ppgn/master/misc/map_clsloc.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24366 (24K) [text/plain]
Saving

In [7]:
import tensorflow as tf
import numpy as np

model = tf.keras.applications.EfficientNetB0(weights='imagenet')

mapping_file = open('map_clsloc.txt',mode='r')

validation_file = open('ILSVRC2012_validation_ground_truth.txt',mode='r')

validation_list = validation_file.readlines()
mapping_list_from = mapping_file.readlines()
mapping_list_to = []
for i in range(len(mapping_list_from)):
  mapping_list_to.append(mapping_list_from[i].split(" ")[1])
  mapping_list_from[i] = mapping_list_from[i].split(" ")[0]

for i in range(len(validation_list)):
  validation_list[i] = validation_list[i].split('\n')[0]

In [26]:
correct = 0
top_5 = 0

it = 50000

for i in range(it):
  img_path = f'data/ILSVRC2012_val_{(i+1):08d}.JPEG'
  img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
  x = tf.keras.preprocessing.image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = tf.keras.applications.efficientnet.preprocess_input(x)
  preds = model.predict(x)

  id = tf.keras.applications.imagenet_utils.decode_predictions(preds, top=1)[0][0][0]
  index = mapping_list_from.index(id)

  if mapping_list_to[index] == validation_list[i]:
    correct = correct + 1
    top_5 = top_5 + 1
  else:
    for ii in range(1,5):
      id = tf.keras.applications.imagenet_utils.decode_predictions(preds, top=5)[0][ii][0]
      index = mapping_list_from.index(id)
      if mapping_list_to[index] == validation_list[i]:
        top_5 = top_5 + 1
        break

  if i!=0 and i%500==0:
    print(str(int(i/500))+'%, Top 1:' + str(correct/i) + ' Top 5:' + str(top_5/i))


1%, Top 1:0.718 Top 5:0.928
2%, Top 1:0.72 Top 5:0.925
3%, Top 1:0.7346666666666667 Top 5:0.924
4%, Top 1:0.7305 Top 5:0.9195
5%, Top 1:0.7348 Top 5:0.9196
6%, Top 1:0.7323333333333333 Top 5:0.9176666666666666
7%, Top 1:0.7302857142857143 Top 5:0.918
8%, Top 1:0.73025 Top 5:0.916
9%, Top 1:0.7313333333333333 Top 5:0.9148888888888889
10%, Top 1:0.731 Top 5:0.9136
11%, Top 1:0.7316363636363636 Top 5:0.9125454545454545
12%, Top 1:0.7305 Top 5:0.9141666666666667
13%, Top 1:0.7324615384615385 Top 5:0.9138461538461539
14%, Top 1:0.7331428571428571 Top 5:0.9144285714285715
15%, Top 1:0.7344 Top 5:0.9142666666666667
16%, Top 1:0.734375 Top 5:0.914625
17%, Top 1:0.7351764705882353 Top 5:0.914
18%, Top 1:0.7334444444444445 Top 5:0.9128888888888889
19%, Top 1:0.7343157894736843 Top 5:0.9126315789473685
20%, Top 1:0.7332 Top 5:0.9123
21%, Top 1:0.7338095238095238 Top 5:0.9131428571428571
22%, Top 1:0.732909090909091 Top 5:0.9125454545454545
23%, Top 1:0.7322608695652174 Top 5:0.9121739130434783
24

In [27]:
accuracy = correct/it*100
top_5_accuracy = top_5/it*100
print('Top 1 Accuracy:'+str(accuracy)+'%')
print('Top 5 Accuracy:'+str(top_5_accuracy)+'%')

Top 1 Accuracy:73.038%
Top 5 Accuracy:91.354%
